In [2]:
!pip install -U sacremoses  # biogpt tokenizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 16.5 MB/s eta 0:00:00


In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import re
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm.auto import tqdm
import shap

In [ ]:
/content/drive/MyDrive/데이터 A/test.csv

In [6]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/데이터 A/"
os.listdir(path)
data = pd.read_csv(path+'test.csv')
data.head()

Mounted at /content/drive


,text,readmission_30
0,Left periprosthetic femur fracture,1.0
1,"Primary: Acute otitis media, mastoiditis, Bell...",1.0
2,Elevated LFTsHA stenosisBiliary stricture with...,1.0
3,"Pharyngitis, supraglottitisAcute Kidney Injury...",1.0
4,Chronic distal common bile duct stricture,1.0


In [7]:
data['readmission_30'] = data['readmission_30'].astype('int64')
data['readmission_30'].value_counts()

,count
readmission_30,
0,6232
1,3768


In [8]:
def extract_and_combine_conditions(text):
    parts = text.split('Secondary:')
    primary_conditions = parts[0].replace('Primary:', '').strip()
    secondary_conditions = parts[1].strip() if len(parts) > 1 else ''
    combined_conditions = primary_conditions + ', ' + secondary_conditions if secondary_conditions else primary_conditions
    return combined_conditions

In [9]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\b(patientname|patientid):\s*\S+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = extract_and_combine_conditions(text)
    return text

In [10]:
class ReadmissionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(self.encodings[key][idx]) for key in self.encodings}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

In [11]:
data['clean_text']=data['text'].apply(clean_text)

In [12]:
test_texts, test_labels = data['clean_text'], data['readmission_30']

In [13]:
blue_tokenizer = AutoTokenizer.from_pretrained('Tolerblanc/blue-BERT-Readmission')
bioC_tokenizer = AutoTokenizer.from_pretrained('Tolerblanc/bioClinical-BERT-Readmission')
sapB_tokenizer = AutoTokenizer.from_pretrained('Tolerblanc/sapBERT-Readmission')
bgpt_tokenizer = AutoTokenizer.from_pretrained('Tolerblanc/biogpt_Readmission')
cg_tokenizer = AutoTokenizer.from_pretrained('medicalai/ClinicalBERT')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [14]:
# 토크나이저 사용
blue_test_encodings = blue_tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)
bioC_test_encodings = bioC_tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)
sapB_test_encodings = sapB_tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)
bgpt_test_encodings = bgpt_tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)
cg_test_encodings = cg_tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)

In [15]:
# 데이터셋 생성
blue_test_dataset = ReadmissionDataset(blue_test_encodings, test_labels.tolist())
bioC_test_dataset = ReadmissionDataset(bioC_test_encodings, test_labels.tolist())
sapB_test_dataset = ReadmissionDataset(sapB_test_encodings, test_labels.tolist())
bgpt_test_dataset = ReadmissionDataset(bgpt_test_encodings, test_labels.tolist())
cg_test_dataset = ReadmissionDataset(cg_test_encodings, test_labels.tolist())

In [16]:
blue_test_loader = DataLoader(blue_test_dataset, batch_size=16, shuffle=False)
bioC_test_loader = DataLoader(bioC_test_dataset, batch_size=16, shuffle=False)
sapB_test_loader = DataLoader(sapB_test_dataset, batch_size=16, shuffle=False)
bgpt_test_loader = DataLoader(bgpt_test_dataset, batch_size=16, shuffle=False)
cg_test_loader = DataLoader(cg_test_dataset, batch_size=16, shuffle=False)

In [17]:
blue_model = AutoModelForSequenceClassification.from_pretrained('Tolerblanc/blue-BERT-Readmission', num_labels=data['readmission_30'].nunique())
bioC_model = AutoModelForSequenceClassification.from_pretrained('Tolerblanc/bioClinical-BERT-Readmission', num_labels=data['readmission_30'].nunique())
sapB_model = AutoModelForSequenceClassification.from_pretrained('Tolerblanc/sapBERT-Readmission', num_labels=data['readmission_30'].nunique())
bgpt_model = AutoModelForSequenceClassification.from_pretrained('Tolerblanc/biogpt_Readmission', num_labels=data['readmission_30'].nunique())
cg_model = AutoModelForSequenceClassification.from_pretrained('medicalai/ClinicalBERT', num_labels=data['readmission_30'].nunique())

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/671 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at medicalai/ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# 추론에 GPU사용 시
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
blue_model.to(device)
bioC_model.to(device)
sapB_model.to(device)
bgpt_model.to(device)
cg_model.to(device)

criterion = nn.CrossEntropyLoss()

In [21]:
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score
import torch

def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            loss = criterion(logits, labels)
            total_loss += loss.item()

            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    average_loss = total_loss / len(test_loader)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')

    return average_loss, f1, recall, accuracy, precision


In [ ]:
test_loss, test_f1, test_recall, test_acc, test_prec = evaluate(blue_model, blue_test_loader, criterion, device)
print(':: :: Finetuned Blue-BERT Benchmark  :: :: ')
print(f'Test Loss: {test_loss:.4f}, F1 Score: {test_f1:.4f}, Recall: {test_recall:.4f}, Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}')

:: :: Finetuned Blue-BERT Benchmark  :: :: 
Test Loss: 0.6153, F1 Score: 0.6308, Recall: 0.6666, Accuracy: 0.6666, Precision: 0.6554


In [ ]:
test_loss, test_f1, test_recall, test_acc, test_prec = evaluate(bioC_model, bioC_test_loader, criterion, device)
print(':: :: Finetuned bioClinical-BERT Benchmark  :: :: ')
print(f'Test Loss: {test_loss:.4f}, F1 Score: {test_f1:.4f}, Recall: {test_recall:.4f}, Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}')

:: :: Finetuned bioClinical-BERT Benchmark  :: :: 
Test Loss: 0.6189, F1 Score: 0.6568, Recall: 0.6654, Accuracy: 0.6654, Precision: 0.6554


In [ ]:
test_loss, test_f1, test_recall, test_acc, test_prec = evaluate(sapB_model, sapB_test_loader, criterion, device)
print(':: :: Finetuned Sap-BERT Benchmark  :: :: ')
print(f'Test Loss: {test_loss:.4f}, F1 Score: {test_f1:.4f}, Recall: {test_recall:.4f}, Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}')

:: :: Finetuned Sap-BERT Benchmark  :: :: 
Test Loss: 0.6213, F1 Score: 0.6688, Recall: 0.6816, Accuracy: 0.6816, Precision: 0.6710


In [ ]:
test_loss, test_f1, test_recall, test_acc, test_prec = evaluate(bgpt_model, bgpt_test_loader, criterion, device)
print(':: :: Finetuned biogpt Benchmark  :: :: ')
print(f'Test Loss: {test_loss:.4f}, F1 Score: {test_f1:.4f}, Recall: {test_recall:.4f}, Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}')

:: :: Finetuned biogpt Benchmark  :: :: 
Test Loss: 0.6278, F1 Score: 0.5854, Recall: 0.6532, Accuracy: 0.6532, Precision: 0.6510


In [ ]:
test_loss, test_f1, test_recall, test_acc, test_prec = evaluate(cg_model, cg_test_loader, criterion, device)
print(':: :: [Control Group] clinicalBERT Benchmark  :: :: ')
print(f'Test Loss: {test_loss:.4f}, F1 Score: {test_f1:.4f}, Recall: {test_recall:.4f}, Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}')

:: :: [Control Group] clinicalBERT Benchmark  :: :: 
Test Loss: 0.6985, F1 Score: 0.2313, Recall: 0.3844, Accuracy: 0.3844, Precision: 0.5829


In [22]:
test_loss, test_f1, test_recall, test_acc, test_prec = evaluate(sapB_model, sapB_test_loader, criterion, device)
print(':: :: Finetuned Sap-BERT Benchmark  :: :: ')
print(f'Test Loss: {test_loss:.4f}, F1 Score: {test_f1:.4f}, Recall: {test_recall:.4f}, Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}')

:: :: Finetuned Sap-BERT Benchmark  :: :: 
Test Loss: 0.6213, F1 Score: 0.6688, Recall: 0.6816, Accuracy: 0.6816, Precision: 0.6710


In [74]:
# Define a function to get model predictions in a format compatible with SHAP
def predict(texts):
    sapB_model.eval()
    if isinstance(texts, str):
        texts = [texts]  # Ensure input is a list
    encoded_inputs = sapB_tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in encoded_inputs.items()}
    with torch.no_grad():
        outputs = sapB_model(**inputs)
        logits = outputs.logits.cpu().numpy()
        probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
    return probabilities

# Fixing the np.exp issue by replacing np.exp with np.exp(logits, out=logits) to avoid axis parameter

def fixed_predict(texts):
    sapB_model.eval()
    if isinstance(texts, str):
        texts = [texts]  # Ensure input is a list
    encoded_inputs = sapB_tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in encoded_inputs.items()}
    with torch.no_grad():
        outputs = sapB_model(**inputs)
        logits = outputs.logits.cpu().numpy()
        exp_logits = np.exp(logits, out=logits)  # Using out parameter to replace axis argument
        probabilities = exp_logits / exp_logits.sum(keepdims=True)
    return probabilities

# Initialize SHAP explainer
explainer = shap.Explainer(lambda x: fixed_predict(x.tolist()), sapB_tokenizer)

# Select a few samples to explain
samples_to_explain = test_texts.iloc[:10].tolist()

# Compute SHAP values
shap_values = explainer(samples_to_explain)

# Visualize explanations for one instance
for i, shap_val in enumerate(shap_values):
    print(f"\nExplanation for sample {i+1}")
    shap.plots.text(shap_val)


Explanation for sample 1



Explanation for sample 2



Explanation for sample 3



Explanation for sample 4



Explanation for sample 5



Explanation for sample 6



Explanation for sample 7



Explanation for sample 8



Explanation for sample 9



Explanation for sample 10


In [77]:
def predict(texts):
    sapB_model.eval()
    if isinstance(texts, str):
        texts = [texts]  # 입력이 리스트가 아닌 경우 리스트로 변환
    encoded_inputs = sapB_tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in encoded_inputs.items()}
    with torch.no_grad():
        outputs = sapB_model(**inputs)
        logits = outputs.logits.cpu().numpy()
        probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
    return probabilities

def fixed_predict(texts):
    sapB_model.eval()
    if isinstance(texts, str):
        texts = [texts]  # 입력이 리스트가 아닌 경우 리스트로 변환
    encoded_inputs = sapB_tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in encoded_inputs.items()}
    with torch.no_grad():
        outputs = sapB_model(**inputs)
        logits = outputs.logits.cpu().numpy()
        exp_logits = np.exp(logits)  # np.exp 사용
        probabilities = exp_logits / exp_logits.sum(axis=1, keepdims=True)
    return probabilities

explainer = shap.Explainer(lambda x: fixed_predict(x.tolist()), sapB_tokenizer)
samples_to_explain = test_texts.iloc[:10].tolist()
shap_values = explainer(samples_to_explain)


PartitionExplainer explainer: 11it [00:10, 10.35s/it]                


In [103]:
shap.plots.text(shap_values[4])